In [2]:
import pandas as pd
from functools import reduce
import statsmodels.api as sm
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [3]:
df = pd.read_csv("data/IL_public_financials.csv").iloc[:, 1:]


In [54]:
df.columns

Index(['UNITID', 'F1A01', 'F1A31', 'F1A04', 'F1A05', 'F1A06', 'F1A19', 'F1A07',
       'F1A08', 'F1A09',
       ...
       'F1C137', 'F1C143', 'F1C144', 'F1C145', 'F1C146', 'F1C147', 'F1C194',
       'F1C195', 'F1C196', 'F1C197'],
      dtype='object', length=213)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Columns: 213 entries, UNITID to F1C197
dtypes: float64(211), int64(2)
memory usage: 199.8 KB


In [45]:
df.describe()

,UNITID,F1A01,F1A31,F1A04,F1A05,F1A06,F1A19,F1A07,F1A08,F1A09,...,F1C137,F1C143,F1C144,F1C145,F1C146,F1C147,F1C194,F1C195,F1C196,F1C197
count,120.000000,1.200000e+02,1.200000e+02,1.200000e+02,1.200000e+02,1.200000e+02,4.800000e+01,1.200000e+02,1.200000e+02,1.200000e+02,...,72.000000,72.000000,72.000000,7.200000e+01,7.200000e+01,72.000000,7.200000e+01,7.200000e+01,60.0,7.200000e+01
mean,146970.083333,1.751844e+08,4.690484e+08,2.064554e+08,6.755037e+08,8.506882e+08,8.965619e+06,1.435527e+07,7.676907e+07,9.112434e+07,...,1216.111111,8228.277778,1020.861111,2.212889e+06,7.807019e+04,56592.638889,3.016140e+07,2.076288e+08,0.0,9.770384e+06
std,1897.129918,2.208185e+08,5.137410e+08,3.603997e+08,8.614365e+08,1.068186e+09,1.624542e+07,2.109974e+07,1.139461e+08,1.320075e+08,...,4657.639037,60564.316178,8662.293772,3.789417e+06,6.624474e+05,206932.446355,3.890305e+07,2.748447e+08,0.0,1.234331e+07
min,144005.000000,1.266330e+07,7.499468e+07,6.947540e+05,7.808452e+07,1.270670e+08,8.865700e+04,9.952840e+05,7.679915e+06,9.415283e+06,...,0.000000,0.000000,0.000000,-1.000000e+00,0.000000e+00,0.000000,2.921072e+06,1.764922e+07,0.0,2.998570e+05
25%,145534.000000,4.879854e+07,1.442073e+08,1.035440e+07,1.560840e+08,1.946568e+08,6.931360e+05,2.560824e+06,1.270680e+07,1.541103e+07,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,5.772444e+06,4.233127e+07,0.0,2.145287e+06
50%,146758.000000,8.212940e+07,2.915057e+08,5.942610e+07,3.072672e+08,3.858595e+08,1.821222e+06,8.015000e+06,2.842986e+07,3.637271e+07,...,0.000000,0.000000,0.000000,4.331150e+04,0.000000e+00,0.000000,1.685560e+07,7.857365e+07,0.0,4.461685e+06
75%,148796.000000,1.383680e+08,4.662964e+08,1.716197e+08,6.188613e+08,7.397228e+08,4.385468e+06,1.104869e+07,5.933023e+07,7.077655e+07,...,0.000000,0.000000,0.000000,3.231373e+06,0.000000e+00,0.000000,2.510213e+07,1.859433e+08,0.0,7.484650e+06
max,149772.000000,7.925552e+08,2.060748e+09,1.713685e+09,3.774432e+09,4.460487e+09,4.950008e+07,1.393783e+08,4.600524e+08,4.974568e+08,...,25217.000000,513040.000000,73502.000000,1.461944e+07,5.621053e+06,902259.000000,1.601407e+08,8.988525e+08,0.0,4.192116e+07


In [50]:
df.head(5)

,UNITID,F1A01,F1A31,F1A04,F1A05,F1A06,F1A19,F1A07,F1A08,F1A09,...,F1C137,F1C143,F1C144,F1C145,F1C146,F1C147,F1C194,F1C195,F1C196,F1C197
0,144005,35060064.0,1.426332e+08,3.189115e+06,1.458223e+08,1.808824e+08,323727.0,2382394.0,9388431.0,11770825.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,144892,57146726.0,2.461221e+08,1.046999e+07,2.565921e+08,3.137388e+08,700265.0,7046883.0,11461487.0,18508370.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,145336,77798800.0,1.060743e+08,6.134302e+06,1.122086e+08,1.900074e+08,299351.0,2030956.0,10961653.0,12992609.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,145600,765329438.0,1.200789e+09,1.064048e+09,2.264837e+09,3.030166e+09,44436961.0,37404387.0,460052401.0,497456788.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,145637,686054869.0,2.060748e+09,1.713685e+09,3.774432e+09,4.460487e+09,42871539.0,72154435.0,413982220.0,486136655.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
unique_ids = list(df['UNITID'].unique())
unique_ids

[144005,
 144892,
 145336,
 145600,
 145637,
 145813,
 147703,
 147776,
 148654,
 149222,
 149231,
 149772]

In [12]:
# separate schools out for imputation
CSU = df[df['UNITID'] == 144005]
EIU =df[df['UNITID'] == 144892]
GSU =df[df['UNITID'] == 145336]
UIC =df[df['UNITID'] == 145600]
UofI =df[df['UNITID'] == 145637]
ISU =df[df['UNITID'] == 145813]
NIU =df[df['UNITID'] == 147703]
Northeastern =df[df['UNITID'] == 147776]
UISpringfield = df[df['UNITID'] == 148654]
SIU = df[df['UNITID'] == 149222]
SIUedwardsville = df[df['UNITID'] == 149231]
WIU = df[df['UNITID'] == 149772]  


In [13]:
CSU.set_index('UNITID')
EIU.set_index('UNITID')
GSU.set_index('UNITID')
UIC.set_index('UNITID')
UofI.set_index('UNITID')
ISU.set_index('UNITID')
NIU.set_index('UNITID')
Northeastern.set_index('UNITID')
UISpringfield.set_index('UNITID')
SIU.set_index('UNITID')
SIUedwardsville.set_index('UNITID')
WIU.set_index('UNITID')

,F1A01,F1A31,F1A04,F1A05,F1A06,F1A19,F1A07,F1A08,F1A09,F1A10,...,F1C137,F1C143,F1C144,F1C145,F1C146,F1C147,F1C194,F1C195,F1C196,F1C197
UNITID,,,,,,,,,,,,,,,,,,,,,
149772,62202490.0,195579146.0,10377562.0,205956708.0,268159198.0,1330068.0,7438493.0,23205196.0,30643689.0,61099625.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149772,66286434.0,206005092.0,10812699.0,216817791.0,283104225.0,1464745.0,7251063.0,18514253.0,25765316.0,68538118.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149772,45733348.0,219337383.0,10723198.0,230060581.0,275793929.0,1222965.0,7106194.0,25706004.0,32812198.0,75786911.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149772,56105710.0,231325715.0,11275811.0,242601526.0,298707236.0,1374100.0,7049753.0,31275957.0,38325710.0,82740778.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149772,86483309.0,250534980.0,14076714.0,264611694.0,351095003.0,NaN,6198276.0,28586135.0,34784411.0,89039091.0,...,0.0,0.0,0.0,356019.0,0.0,0.0,17363656.0,64782998.0,0.0,4580849.0
149772,84485888.0,220733838.0,52219268.0,272953106.0,357438994.0,NaN,5944530.0,29753458.0,35697988.0,95113458.0,...,0.0,0.0,0.0,107725.0,0.0,0.0,15390832.0,60497374.0,0.0,4754232.0
149772,98571023.0,190411113.0,57666476.0,248077589.0,346648612.0,NaN,5613335.0,34961317.0,40574652.0,100117691.0,...,0.0,0.0,0.0,2674002.0,0.0,0.0,13263756.0,62314745.0,0.0,4900309.0
149772,124385381.0,153534004.0,61185734.0,214719738.0,339105119.0,NaN,5617804.0,28273585.0,33891389.0,105731026.0,...,0.0,0.0,0.0,290137.0,0.0,60.0,11632252.0,58976636.0,0.0,4535369.0
149772,119213904.0,139732597.0,37213001.0,176945598.0,296159502.0,NaN,4329393.0,29884875.0,34214268.0,84903444.0,...,0.0,0.0,0.0,42407.0,0.0,0.0,10849182.0,55977621.0,0.0,4042482.0


In [14]:
imp_mean = SimpleImputer(missing_values=np.nan)
imp_mean.fit_transform(CSU)
imp_mean.fit_transform(EIU)
imp_mean.fit_transform(GSU)
imp_mean.fit_transform(UIC)
imp_mean.fit_transform(UofI)
imp_mean.fit_transform(ISU)
imp_mean.fit_transform(NIU)
imp_mean.fit_transform(Northeastern)
imp_mean.fit_transform(UISpringfield)
imp_mean.fit_transform(SIU)
imp_mean.fit_transform(SIUedwardsville)
imp_mean.fit_transform(WIU)


array([[1.49772000e+05, 6.22024900e+07, 1.95579146e+08, ...,
        5.99813782e+07, 0.00000000e+00, 4.10120183e+06],
       [1.49772000e+05, 6.62864340e+07, 2.06005092e+08, ...,
        5.99813782e+07, 0.00000000e+00, 4.10120183e+06],
       [1.49772000e+05, 4.57333480e+07, 2.19337383e+08, ...,
        5.99813782e+07, 0.00000000e+00, 4.10120183e+06],
       ...,
       [1.49772000e+05, 1.24385381e+08, 1.53534004e+08, ...,
        5.89766360e+07, 0.00000000e+00, 4.53536900e+06],
       [1.49772000e+05, 1.19213904e+08, 1.39732597e+08, ...,
        5.59776210e+07, 0.00000000e+00, 4.04248200e+06],
       [1.49772000e+05, 8.21186800e+07, 1.34526052e+08, ...,
        5.73388950e+07, 0.00000000e+00, 1.79397000e+06]])

In [11]:
# #drop any column that has almost 40% missing values
# res2 = df.columns[df.isnull().sum() > 428]
# df.drop(res2, inplace=True, axis=1)
# df.set_index(['year', 'UNITID'], inplace=True)

In [13]:
df.dropna(inplace=True)

In [15]:
df.isna().sum().sum()

4758

In [5]:
# #Getting rid of Governors State, because it is missing graduation rates
# df = df[df["UNITID"] != 145336]

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 616 entries, (2018, 144005) to (2010, 243780)
Columns: 134 entries, F1A01 to F1EQUITR
dtypes: float64(134)
memory usage: 652.4 KB


In [16]:
df.describe()

,F1A01,F1A31,F1A04,F1A05,F1A06,F1A07,F1A08,F1A09,F1A10,F1A11,...,F1RSRCFT,F1PBSVFT,F1ACSPFT,F1STSVFT,F1INSUFT,F1OTEXFT,F1SAFBPC,F1SALRPC,F1ENDMFT,F1EQUITR
count,6.160000e+02,6.160000e+02,6.160000e+02,6.160000e+02,6.160000e+02,6.160000e+02,6.160000e+02,6.160000e+02,6.160000e+02,6.160000e+02,...,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000,616.000000
mean,2.152246e+08,5.742421e+08,4.280420e+08,1.002284e+09,1.217509e+09,2.415965e+07,8.921097e+07,1.133706e+08,2.453960e+08,1.296501e+08,...,2266.928571,1331.199675,2767.639610,1864.134740,2290.019481,1589.443182,61.655844,44.496753,12981.366883,62.801948
std,4.287443e+08,9.174326e+08,1.339564e+09,2.190520e+09,2.582298e+09,7.515903e+07,1.910140e+08,2.509113e+08,3.861804e+08,4.580587e+08,...,4394.797865,1501.226487,1993.407723,824.133447,1138.114534,1282.135038,5.957988,4.833823,27543.669892,17.296639
min,0.000000e+00,4.486472e+06,-1.488284e+07,8.407980e+06,1.069858e+07,0.000000e+00,9.126360e+05,9.126360e+05,0.000000e+00,0.000000e+00,...,0.000000,1.000000,362.000000,641.000000,477.000000,-4474.000000,35.000000,32.000000,6.000000,3.000000
25%,3.617142e+07,1.206419e+08,1.392201e+07,1.494986e+08,1.856089e+08,2.482500e+06,9.523132e+06,1.296518e+07,4.293171e+07,1.885359e+06,...,124.750000,402.500000,1764.000000,1198.000000,1552.250000,992.750000,58.000000,41.000000,2427.250000,55.000000
50%,7.252399e+07,2.468834e+08,6.224519e+07,3.169247e+08,3.833925e+08,5.999216e+06,2.409276e+07,3.057562e+07,9.797221e+07,1.455527e+07,...,524.000000,771.500000,2177.500000,1675.500000,2178.000000,1318.500000,62.000000,44.500000,4705.000000,64.000000
75%,1.974072e+08,6.003848e+08,2.082577e+08,8.323312e+08,1.018642e+09,1.371275e+07,7.278458e+07,8.682993e+07,2.824267e+08,6.675163e+07,...,1846.750000,1559.250000,3066.000000,2372.750000,2852.000000,1829.500000,66.000000,48.000000,10522.000000,73.000000
max,4.056535e+09,5.886034e+09,1.264431e+10,1.853034e+10,2.116076e+10,6.592285e+08,1.524150e+09,2.180885e+09,2.729577e+09,5.320828e+09,...,26727.000000,9712.000000,20210.000000,5201.000000,14850.000000,10100.000000,83.000000,65.000000,249867.000000,98.000000


In [17]:
# nan_values = df.isna().sum()
# nan_columns = nan_values.any()
# columns_with_nan = df.columns[nan_columns].tolist()
# print(columns_with_nan)                                    

[]


In [18]:
import statsmodels.api as sm
			
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            #if verbose:
                #print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            #if verbose:
                #print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    included.append('id')
    print('resulting features:')
    print(included)
    
    return included

In [19]:
#set target graduation rates
y = df['GBA6RTBK']
X = df.drop(columns=['GBA6RTBK'], axis=1)
X

F1A01         F1A31         F1A04         F1A05  \
year UNITID                                                          
2018 144005   38819933.0  1.461852e+08  3.213347e+06  1.493986e+08   
     144892   58718575.0  2.558635e+08  1.087340e+07  2.667369e+08   
     145600  792555178.0  1.197650e+09  8.805430e+08  2.078193e+09   
     145637  663042196.0  2.015473e+09  1.412279e+09  3.427752e+09   
     145813  141321777.0  4.418369e+08  2.457655e+08  6.876024e+08   
...                  ...           ...           ...           ...   
2010 240453  143670091.0  4.292438e+08  2.192314e+07  4.511669e+08   
     240462   35587940.0  1.560059e+08  8.288773e+06  1.642947e+08   
     240471   30131893.0  1.215047e+08  5.065807e+06  1.265705e+08   
     240480   53034467.0  1.725112e+08  1.244722e+07  1.849584e+08   
     243780  639287972.0  1.269845e+09  1.960896e+09  3.230741e+09   

                    F1A06        F1A07        F1A08        F1A09        F1A10  \
year UNITID                                                                     
2018 144005  1.882185e+08    2300799.0   10628649.0   12929448.0   11407787.0   
     144892  3.254555e+08    6959076.0   11508992.0   18468068.0   86430825.0   
     145600  2.870748e+09   35145081.0  382215290.0  417360371.0  552319383.0   
     145637  4.090794e+09   61841665.0  380901626.0  442743291.0  788771731.0   
     145813  8.289242e+08    8857361.0   36191836.0   45049197.0  154696415.0   
...                   ...          ...          ...          ...          ...   
2010 240453  5.948370e+08    2734228.0   49181973.0   51916201.0   41372350.0   
     240462  1.998826e+08    3039662.0    8217792.0   11257454.0   64952944.0   
     240471  1.567024e+08    1497915.0    8937699.0   10435614.0   30527856.0   
     240480  2.379929e+08    3058195.0   18226278.0   21284473.0   62526718.0   
     243780  3.870029e+09  144506654.0  330067637.0  474574291.0  500554273.0   

                   F1A11  ...  F1RSRCFT  F1PBSVFT  F1ACSPFT  F1STSVFT  \
year UNITID               ...                                           
2018 144005    7272999.0  ...    1088.0    1957.0    4889.0    3665.0   
     144892   15946190.0  ...     185.0     832.0    2845.0    3883.0   
     145600  759796964.0  ...    9447.0    8212.0    7482.0    2428.0   
     145637  774422853.0  ...   10435.0    4753.0    8281.0    3413.0   
     145813   51922223.0  ...     780.0     804.0    1398.0    2102.0   
...                  ...  ...       ...       ...       ...       ...   
2010 240453    5751860.0  ...    2414.0     843.0    1831.0    2200.0   
     240462     837110.0  ...     100.0     477.0    1916.0    2051.0   
     240471     856501.0  ...      73.0     427.0    1920.0    2368.0   
     240480    1266785.0  ...     602.0    1610.0    1533.0    2205.0   
     243780   67616805.0  ...    5793.0    3619.0    1242.0     805.0   

             F1INSUFT  F1OTEXFT  F1SAFBPC  F1SALRPC  F1ENDMFT  F1EQUITR  
year UNITID                                                              
2018 144005    9404.0    3800.0      72.0      39.0    2329.0      79.0  
     144892    1945.0    1741.0      74.0      37.0    9683.0      63.0  
     145600    2523.0    2463.0      66.0      35.0   10266.0      36.0  
     145637    1750.0    4055.0      60.0      33.0   30641.0      48.0  
     145813    2605.0   10100.0      35.0      34.0    6761.0      69.0  
...               ...       ...       ...       ...       ...       ...  
2010 240453    1075.0    1314.0      67.0      48.0    2866.0      83.0  
     240462    1008.0    1323.0      59.0      44.0    1694.0      61.0  
     240471    1296.0    1326.0      61.0      43.0    1544.0      73.0  
     240480    1287.0    1878.0      60.0      42.0    1901.0      64.0  
     243780    3498.0    1426.0      66.0      46.0   39139.0      73.0  

[616 rows x 133 columns]

In [20]:

stepwise_selection(X,y)

<ipython-input-18-59576973df7c>:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-18-59576973df7c>:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-18-59576973df7c>:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-18-59576973df7c>:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-18-59576973df7c>:26: 

resulting features:
['F1FHA', 'F1B05', 'F1GVGCPC', 'F1RSRCFT', 'F1EQUITR', 'F1E01', 'F1STAPPC', 'F1C061', 'F1COREXP', 'F1E10', 'F1B13', 'F1E06', 'id']


['F1FHA',
 'F1B05',
 'F1GVGCPC',
 'F1RSRCFT',
 'F1EQUITR',
 'F1E01',
 'F1STAPPC',
 'F1C061',
 'F1COREXP',
 'F1E10',
 'F1B13',
 'F1E06',
 'id']